### Introducción del proyecto 

El proyecto tiene como objetivo predecir si un cliente de Beta Bank dejará o no el banco en un futuro cercano, utilizando un conjunto de datos que describe diversas características de los clientes, como su saldo, edad, puntuación crediticia, y si son miembros activos, entre otras caracteristicas. Debido a que el conjunto de datos presenta un desbalance en las clases (una gran mayoría de clientes no abandona el banco), el desafío principal es desarrollar un modelo que logre predecir correctamente el abandono de clientes, utilizando métricas como el F1 Score y AUC-ROC para evaluar su desempeño. Para esto, se emplean técnicas de ajuste de modelos, corrección de desequilibrio de clases y la selección de hiperparámetros adecuados para optimizar el rendimiento del modelo.

### Desarrollo:
#### Paso 1: Cargar librerías y explorar dataset


In [1]:
#carga de librerías
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score 

In [2]:
clients = pd.read_csv("/datasets/Churn.csv")

In [3]:
# se usan las funciones info y head para inspección de la información  del dataset
clients.info()
print(clients.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    1563

In [4]:
#revisión de filas duplicadas
print(clients.duplicated().sum())
print()
#revisión valores ausentes 
print(clients.isna().sum())

0

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64


In [5]:
#se ve cantidad de valores unicos en columnas para ver la factibilidad de conversión de columnas categóricas
print(clients.nunique())

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64


In [6]:
# se completan valores faltantes en 'Tenure' con valor medio de la columna
clients['Tenure'].fillna(clients['Tenure'].mean(), inplace=True)
# se hace una conversión de columnas categóricas
clients = pd.get_dummies(clients, columns=['Geography', 'Gender'], drop_first=True)

#### Paso 2: Dividir el dataset en conjuntos de entrenamiento, validación y prueba

El conjunto de prueba no existe, por lo que los datos fuente deben dividirse en tres partes: entrenamiento, validación y prueba. Usualmente, el tamaño del conjunto de validación y del de prueba son iguales. Esto da como resultado una proporción de datos fuente de 3:1:1

In [7]:
# Primer paso: se separan las características y la variable objetivo
features = clients.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)  # se eliminan columnas innecesarias
target = clients['Exited']

In [8]:
# Segundo paso: dividir en conjunto de entrenamiento (60%) y conjunto combinado de prueba/validación (40%)
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.4, random_state=12345)

# Tercer paso: dividir el 40% en conjunto de prueba (50%) y conjunto de validación (50%)
features_test, features_valid, target_test, target_valid = train_test_split(features_temp, target_temp, test_size=0.5, random_state=12345)

#### Paso 3: Entrenamiento Inicial del Modelo
##### Entrenar un Modelo Sin Corregir el Desequilibrio

Inicialmente entrenamos dos modelos sin aplicar ninguna corrección al desequilibrio de clases: Regresión Logística y Árbol de Decisión. El objetivo es obtener una referencia base del rendimiento de ambos modelos utilizando las métricas F1 y AUC-ROC, que nos permitirán evaluar la capacidad de cada modelo para predecir correctamente el abandono de clientes. Tras obtener los resultados, seleccionaremos el modelo que muestre el mejor desempeño en estas métricas para realizar ajustes posteriores y mejorar su capacidad de predicción.

1. Regresión Logística

In [9]:
# Prueba de modelo de regresión logística
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)

# Predecir en el conjunto de prueba
target_pred = model.predict(features_test)

# Calculo F1 y AUC-ROC
f1 = f1_score(target_test, target_pred)
auc_roc = roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

print(f"F1 Score: {f1}")
print(f"AUC-ROC: {auc_roc}")

F1 Score: 0.08385744234800838
AUC-ROC: 0.6727947180904797


2. Árbol de decisión

In [10]:
# Prueba de modelo de árbol de desición 
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)

# Predecir en el conjunto de prueba
target_pred = model.predict(features_test)

# Calculo F1 y AUC-ROC
f1 = f1_score(target_test, target_pred)
auc_roc = roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

print(f"F1 Score: {f1}")
print(f"AUC-ROC: {auc_roc}")

F1 Score: 0.5012165450121654
AUC-ROC: 0.683832469347141


##### Hallazgos Iniciales
Después de entrenar los modelos, observamos los valores de F1 y AUC-ROC para evaluar su desempeño. En el caso de la Regresión Logística, los resultados fueron bastante bajos, con un F1 Score de 0.083 y un AUC-ROC de 0.673, lo que indica que el modelo no está logrando capturar de manera efectiva las clases minoritarias, probablemente debido al desequilibrio de clases. Por otro lado, el Árbol de Decisión mostró un rendimiento más prometedor, con un F1 Score de 0.501 y un AUC-ROC de 0.684. Aunque el F1 del árbol es significativamente mejor, sigue siendo insuficiente para un modelo optimizado. Debido a su rendimiento superior, se decide proceder con el Árbol de Decisión para realizar los ajustes necesarios y abordar el desequilibrio de clases.

#### Paso 4: Mejorar el Modelo

En esta etapa primero se observa la distribución de clases en el conjunto de entrenamiento:

In [11]:
# Ver la distribución de clases
target_train.value_counts(normalize=True)

0    0.800667
1    0.199333
Name: Exited, dtype: float64

Esto indica que el conjunto de datos está desequilibrado, con aproximadamente el 80% de los clientes que no se han ido del banco y el 20% que sí se han ido. Para corregir esto se va a entrenar el modelo con el parámetro class_weight='balanced' para ajusta el peso de cada clase para equilibrar su influencia en el modelo. A continuación, tambien se buscara la mejor profundidad del árbol de decisión para optimizar el modelo.

In [12]:
best_depth = None
best_accuracy = 0

# se prueban diferentes profundidades
for depth in range(1, 8):
        model = DecisionTreeClassifier(random_state=12345,class_weight='balanced', max_depth=depth)
        model.fit(features_train, target_train)

        predictions_valid = model.predict(features_valid)
        # se calcula la exactitud en el conjunto de validación
        accuracy = accuracy_score(target_valid, predictions_valid)

        print("max_depth =", depth, ": ", end='')
        print(accuracy)
        
        # se guarda la mejor puntuación de accuracy y depth en el conjunto de validación
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_model = model # se guarda el mejor modelo
        

max_depth = 1 : 0.74
max_depth = 2 : 0.7465
max_depth = 3 : 0.7465
max_depth = 4 : 0.7155
max_depth = 5 : 0.798
max_depth = 6 : 0.762
max_depth = 7 : 0.77


Como se puede observar, la mejor exactitud se obtiene con max_depth = 5, por lo que se selecciona esta profundidad para el modelo.

A continuación, se entrena el modelo final con la profundidad escogida y se evalúa en el conjunto de prueba:

In [13]:
# se ajusta peso de clase en el modelo añadiendo class_weight y se agrega profundidad adecuada
# se entrena el modelo con los parametros incorporados
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced', max_depth = 5 )
model.fit(features_train, target_train)

# Predecir en el conjunto de prueba
target_pred = model.predict(features_test)

# Calcular F1 y AUC-ROC
f1 = f1_score(target_test, target_pred)
auc_roc = roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

print(f"F1 Score: {f1}")
print(f"AUC-ROC: {auc_roc}")

F1 Score: 0.5963791267305644
AUC-ROC: 0.8310244134068074


### Conclusión
A través de este proyecto, se logró desarrollar un modelo efectivo para predecir el abandono de clientes en Beta Bank, enfrentando el desafío del desbalance de clases en los datos. Inicialmente, se entrenó un modelo de regresión logística y un árbol de decisión sin considerar el desequilibrio, lo que resultó en un bajo rendimiento. Posteriormente, se implementaron diversas técnicas para manejar el desbalance, como el uso de class_weight='balanced', y se ajustaron los parámetros del modelo, como la profundidad máxima en un árbol de decisión, obteniendo finalmente un F1 Score de 0.596 y un AUC-ROC de 0.83, lo que supera el umbral mínimo requerido. Estos resultados muestran que el modelo es capaz de identificar con mayor precisión a los clientes en riesgo de abandonar el banco, lo cual es crucial para que el banco pueda enfocarse en retener a estos clientes de manera más eficiente y rentable.